## Armor Skills

### Naming Scheme:

**Attack Boost 2: ATK +5**

| Attack Boost | 2 | ATK +5 |
|-|-|-|
| Common | Lvl | Descript. |

- sc_id (skill common id) : id for skill independent of level
- skill_id : id for specific skill & level
- skill_category : weapon, equipment, group, etc.
- skill_lv : level of skill


In [167]:
# head cell
import urllib.request
import json
import pandas as pd
import numpy as np
import re

### Common Skill IDs, Names, & Descriptions
#### Source
natives/STM/GameDesign/Text/Excel_Equip/SkillCommon.msg.23.json
```
"entries": [
    "name": "SkillCommon_m1961116288",
    "content": [
        "Rapid Morph",
    "name": "SkillCommon_EXPm1961116288",
    "content": [
        "Increases switch speed and power\r\nfor switch axes and charge blades.",
```

#### df_skill_common_ids
|common_id|guid_n|name|guid_d|description|
|-|-|-|-|-|
|SkillCommon_0|82d0863d-8ea5-4106-9392-a8b7575f6b9f|SkillCommon_0|ab11e7f5-8b70-4dff-a0ac-f9643ef55e6a|SkillCommon_EXP0|
|SkillCommon_1|c5243424-047c-40cf-a6f4-69e9d3ae860f|Attack Boost|38b4b21e-d241-4c08-ae35-cff0632574e2|Increases attack power.|
|SkillCommon_m2096489472|c4dc2439-1d42-4ef4-bca9-07be35f4bebd|Critical Eye|768c9e10-dfe1-4373-937c-d6376e92a26a|Increases affinity.|
|SkillCommon_m1607763456|6e7e9e83-16b1-4bf6-92e9-98eff98d85ac|Critical Boost|671c0922-1fe5-4c78-aac4-213ea807daa1|Increases the damage of critical hits.|
|SkillCommon_313598432|b09b9e70-7243-45c5-86e3-158ed24004e8|Critical Element|f1506057-3237-400b-a7aa-4f6619236f11|Increases elemental damage (fire, water, thunder, ice, dragon) when landing critical hits.|

#### Dicts
- dict_common_guid_names { guid_n : name }
- dict_common_guid_desc { guid_d : description }
- dict_common_id_names { sc_id : name }

In [168]:
url = 'https://raw.githubusercontent.com/dtlnor/MHWs-in-json/refs/heads/main/natives/STM/GameDesign/Text/Excel_Equip/SkillCommon.msg.23.json'

with urllib.request.urlopen(url) as page:
    raw = json.load(page)
sect = raw.get('entries')
df = pd.json_normalize(sect)

# pull english names
df['content'] = df['content'].str[1]

#separate names and descriptions
df_1 = df[~df['name'].str.contains('EXP')].copy()
df_2 = df[df['name'].str.contains('EXP')].copy()

#normalize naming
df_1.rename(columns={'name':'id','content':'name', 'guid':'guid_n'}, inplace=True)
df_2['name'] = df_2['name'].str.replace('EXP','')
df_2.rename(columns={'name':'id','content':'description', 'guid':'guid_d'}, inplace=True)

#adjust content
df_1['name'] = df_1['name'].str.replace('<COLOR FF0000>#Rejected#</COLOR> ','')
df_2['description'] = df_2['description'].str.replace('<COLOR FF0000>#Rejected#</COLOR> ','')
df_2['description'] = df_2['description'].str.replace(r'[\r\n]+', ' ', regex=True)
df_2['description'] = df_2['description'].str.replace('<BOLD>','')
df_2['description'] = df_2['description'].str.replace('</BOLD>','')

#merge into final
df = pd.merge(df_1[['id','guid_n','name']], df_2[['id','guid_d','description']], on='id', how='left')
df.reset_index(drop=True, inplace=True)

#dicts
dict_common_guid_names = dict(zip(df['guid_n'], df['name']))
dict_common_guid_desc = dict(zip(df['guid_d'], df['description']))
dict_common_id_names = dict(zip(df['id'], df['name']))

#format skill common id
# df['id'] = df['id'].str.replace('SkillCommon_','')
# df['id'] = df['id'].str.replace('m','-')
# df.rename(columns={'id':'skill_common_id'}, inplace=True)

df.rename(columns={'id':'common_id'}, inplace=True)

df_skill_common_ids = df.copy()

df_skill_common_ids
# df_skill_common.to_excel('text skill common.xlsx')

,common_id,guid_n,name,guid_d,description
0,SkillCommon_0,82d0863d-8ea5-4106-9392-a8b7575f6b9f,SkillCommon_0,ab11e7f5-8b70-4dff-a0ac-f9643ef55e6a,SkillCommon_EXP0
1,SkillCommon_1,c5243424-047c-40cf-a6f4-69e9d3ae860f,Attack Boost,38b4b21e-d241-4c08-ae35-cff0632574e2,Increases attack power.
2,SkillCommon_m2096489472,c4dc2439-1d42-4ef4-bca9-07be35f4bebd,Critical Eye,768c9e10-dfe1-4373-937c-d6376e92a26a,Increases affinity.
3,SkillCommon_m1607763456,6e7e9e83-16b1-4bf6-92e9-98eff98d85ac,Critical Boost,671c0922-1fe5-4c78-aac4-213ea807daa1,Increases the damage of critical hits.
4,SkillCommon_313598432,b09b9e70-7243-45c5-86e3-158ed24004e8,Critical Element,f1506057-3237-400b-a7aa-4f6619236f11,"Increases elemental damage (fire, water, thund..."
...,...,...,...,...,...
187,SkillCommon_504506560,5195cade-5aa9-4140-872b-28cdf40d1d00,SkillCommon_504506560,829fa827-1837-49c5-83d0-f414deaf4bf5,SkillCommon_EXP504506560
188,SkillCommon_m1662120192,3a6ba0f4-838a-40b3-ade0-f7496d5b9b8e,Antivirus,1acfbd37-9136-4016-8c4d-18aa187834b4,"Once infected, makes it easier to overcome the..."
189,SkillCommon_m1702725248,1ecaa532-e3fc-488d-80e8-852dfd3486e2,SkillCommon_m1702725248,e7168962-019e-4498-84ae-3acf2e0d2005,SkillCommon_EXPm1702725248
190,SkillCommon_1613139840,fef14cf8-7956-4c08-a70f-6969337f98e0,Tetrad Shot,a802b785-eba1-414e-a775-f209eb02ebc5,Increases affinity of bowgun ammo/ coatings fr...


### Specific Skill IDs & Descriptions

#### Source
natives/STM/GameDesign/Text/Excel_Equip/Skill.msg.23.json
```
"entries": [
    {
        "name": "Skill_488",
      "guid": "95f8239f-d0c1-4120-b3fb-37378e8875f5",
      ...
      "content": [
        "<COLOR FF0000>#Rejected#</COLOR> Skill_488",
        "<COLOR FF0000>#Rejected#</COLOR> Skill_488",]
    },
    {
      "name": "Skill_EXP488",
      "guid": "f72a7bc8-58c8-4550-afc3-2450a5b463c2",
      ...
      "content": [
        "会心攻撃時のダメージ倍率を\r\n１．３７倍に強化する",
        "Increases damage dealt\r\nby critical hits to 37%.",
      ]
    },
```
#### df_skill_spec_ids
|spec_id|guid_n|name|guid_d|description|
|-|-|-|-|-|
|Skill_1|f7457609-c698-44dc-a87c-3013a81fe34f|Skill_1|3b08702a-e4a1-4de4-b7db-f5897fef1c4b|Skill_EXP1|
|Skill_2|824fa7e2-6344-4f5d-a140-0d411ccc674d|Skill_2|96e65c81-c4a1-4fb0-aea4-c256200cda88|Attack +3|
|Skill_3|c3858b4a-0f53-4739-ade5-ddf1c08497ad|Skill_3|b97ead47-2c69-436f-ab9d-4f9059ec151f|Attack +5|
|Skill_4|2a36d1c4-d599-4b2d-8385-47ebef1df235|Skill_4|9d878426-10dd-4506-9259-6548d4c59735|Attack +7|
|Skill_5|85873022-2c20-4f4a-9e13-6c2e1275918a|Skill_5|86a03b9f-c136-414c-8fe5-501b6c4b6b73|Attack +2% Bonus: +8|

#### Dicts
- dict_spec_guid_names: { guid_n : name }
- dict_spec_guid_desc: { guid_d : description }

In [169]:
url = 'https://raw.githubusercontent.com/dtlnor/MHWs-in-json/refs/heads/main/natives/STM/GameDesign/Text/Excel_Equip/Skill.msg.23.json'

with urllib.request.urlopen(url) as page:
    raw = json.load(page)
sect = raw.get('entries')
df = pd.json_normalize(sect)

# pull english names
df['content'] = df['content'].str[1]

#separate names and descriptions
df_1 = df[~df['name'].str.contains('EXP')].copy()
df_2 = df[df['name'].str.contains('EXP')].copy()

#normalize naming
df_1.rename(columns={'name':'id','content':'name', 'guid':'guid_n'}, inplace=True)
df_2['name'] = df_2['name'].str.replace('EXP','')
df_2.rename(columns={'name':'id','content':'description', 'guid':'guid_d'}, inplace=True)

#adjust content
# df_1 = df_1[~df_1['name'].str.contains('<COLOR FF0000>')].copy()
# df_2 = df_2[~df_2['description'].str.contains('<COLOR FF0000>')].copy()
df_1['name'] = df_1['name'].str.replace('<COLOR FF0000>#Rejected#</COLOR> ','')
df_2['description'] = df_2['description'].str.replace('<COLOR FF0000>#Rejected#</COLOR> ','')
df_2['description'] = df_2['description'].str.replace(r'[\r\n]+', ' ', regex=True)
df_2['description'] = df_2['description'].str.replace('<BOLD>','')
df_2['description'] = df_2['description'].str.replace('</BOLD>','')


#merge into final
df = pd.merge(df_1[['id','guid_n','name']], df_2[['id','guid_d','description']], on='id', how='left')
df.reset_index(drop=True, inplace=True)

#dicts
dict_spec_guid_names = dict(zip(df['guid_n'], df['name']))
dict_spec_guid_desc = dict(zip(df['guid_d'], df['description']))

#format skill ids
# df['id'] = df['id'].str.replace('Skill_','')
df.rename(columns={'id':'spec_id'}, inplace=True)

df_skill_spec_ids = df.copy()

# df_1.head()
df_skill_spec_ids.head()
# df_skills.to_excel('text skill.xlsx')

,spec_id,guid_n,name,guid_d,description
0,Skill_1,f7457609-c698-44dc-a87c-3013a81fe34f,Skill_1,3b08702a-e4a1-4de4-b7db-f5897fef1c4b,Skill_EXP1
1,Skill_2,824fa7e2-6344-4f5d-a140-0d411ccc674d,Skill_2,96e65c81-c4a1-4fb0-aea4-c256200cda88,Attack +3
2,Skill_3,c3858b4a-0f53-4739-ade5-ddf1c08497ad,Skill_3,b97ead47-2c69-436f-ab9d-4f9059ec151f,Attack +5
3,Skill_4,2a36d1c4-d599-4b2d-8385-47ebef1df235,Skill_4,9d878426-10dd-4506-9259-6548d4c59735,Attack +7
4,Skill_5,85873022-2c20-4f4a-9e13-6c2e1275918a,Skill_5,86a03b9f-c136-414c-8fe5-501b6c4b6b73,Attack +2% Bonus: +8


### Common Skills

#### Source
natives/STM/GameDesign/Common/Equip/SkillCommonData.user.3.json
```
"app.user_data.SkillCommonData": {
    "_Values": [
        {
            "app.user_data.SkillCommonData.cData": {
                "_Index": 4,
                "_skillId": "[-181127504]HunterSkill_028",
                "_skillType": "[441208832]NONE",
                "_skillCategory": "[3]WEAPON",
                "_SkillIconType": "[1]SKILL_0000",
                "_skillName": "b759c7ca-775e-4779-ab53-f94bf2962305",
                "_skillExplain": "7c1468b7-0445-4dc1-acb2-6d5bf228f9f5",
                "_SortId": 4
            }
        },
```

#### df_common_skills
|name|category|description|SkillIconType|common_hunter_id|common_id|hunter_id|SortId|
|-|-|-|-|-|-|-|-|
|SkillCommon_0|[0]EQUIP|SkillCommon_EXP0|[1]SKILL_0000|[0]NONE|SkillCommon_0|NONE|0|
|Attack Boost|[3]WEAPON|Increases attack power.|[1]SKILL_0000|[1]HunterSkill_000|SkillCommon_1|HunterSkill_000|1|
|Resentment|[0]EQUIP|Increases attack when you have recoverable damage (the red portion of your Health Gauge).|[1]SKILL_0000|[1359821952]HunterSkill_060|SkillCommon_1359821952|HunterSkill_060|2|
|Adrenaline Rush|[0]EQUIP|Temporarily increases attack power after executing a perfectly-timed evade just as a monster attacks.|[1]SKILL_0000|[1174975744]HunterSkill_115|SkillCommon_1174975744|HunterSkill_115|3|
|Offensive Guard|[3]WEAPON|Temporarily increases attack power after executing a perfectly-timed guard.|[1]SKILL_0000|[-181127504]HunterSkill_028|SkillCommon_m181127504|HunterSkill_028|4|

#### Dicts
- dict_common_hunter_name: { common_hunter_id : name }

In [170]:
url = 'https://raw.githubusercontent.com/dtlnor/MHWs-in-json/refs/heads/main/natives/STM/GameDesign/Common/Equip/SkillCommonData.user.3.json'

with urllib.request.urlopen(url) as page:
    raw = json.load(page)
df = pd.json_normalize(raw, record_path=['app.user_data.SkillCommonData', '_Values'])
df.columns = df.columns.str.replace('app.user_data.SkillCommonData.cData._','')

#split ids
# df['hs_id'] = df['skillId'].str.extract(r'^(?:.*_)(\d+)$')
df['hunter_id'] = df['skillId'].str.extract(r'^\[[^]]+\](.*)$')
df['common_id'] = 'SkillCommon_' + df['skillId'].str.extract(r'\[(.*?)\]')
df['common_id'] = df['common_id'].str.replace('-','m')

df['skillName'] = df['skillName'].replace(dict_common_guid_names)
df['skillExplain'] = df['skillExplain'].replace(dict_common_guid_desc)

df_common_skills = df[['skillName','skillCategory','skillExplain','SkillIconType','skillId','common_id','hunter_id','SortId']].copy()
df.reset_index(drop=True, inplace=True)

df_common_skills.rename(columns={'skillName':'name',
                   'skillExplain':'description',
                   'skillCategory':'category',
                   'skillId':'common_hunter_id'}, inplace=True)

#dicts
dict_common_hunter_name = dict(zip(df_common_skills['common_hunter_id'],df_common_skills['name']))

df_common_skills
# df_common_skills.to_excel('common_skills.xlsx')

,name,category,description,SkillIconType,common_hunter_id,common_id,hunter_id,SortId
0,SkillCommon_0,[0]EQUIP,SkillCommon_EXP0,[1]SKILL_0000,[0]NONE,SkillCommon_0,NONE,0
1,Attack Boost,[3]WEAPON,Increases attack power.,[1]SKILL_0000,[1]HunterSkill_000,SkillCommon_1,HunterSkill_000,1
2,Resentment,[0]EQUIP,Increases attack when you have recoverable dam...,[1]SKILL_0000,[1359821952]HunterSkill_060,SkillCommon_1359821952,HunterSkill_060,2
3,Adrenaline Rush,[0]EQUIP,Temporarily increases attack power after execu...,[1]SKILL_0000,[1174975744]HunterSkill_115,SkillCommon_1174975744,HunterSkill_115,3
4,Offensive Guard,[3]WEAPON,Temporarily increases attack power after execu...,[1]SKILL_0000,[-181127504]HunterSkill_028,SkillCommon_m181127504,HunterSkill_028,4
...,...,...,...,...,...,...,...,...
187,Rice Ball,[0]EQUIP,Increases the attack power of Akuma actions an...,[9]SKILL_0008,[10503]HunterSkill_211,SkillCommon_10503,HunterSkill_211,375
188,Festival Spirit,[2]GROUP,SkillCommon_EXP13941,[13]SKILL_0012,[13941]HunterSkill_219,SkillCommon_13941,HunterSkill_219,514
189,SkillCommon_414,[0]EQUIP,SkillCommon_EXP414,[0]INVALID,[414]HunterSkill_220,SkillCommon_414,HunterSkill_220,0
190,Glory's Favor,[2]GROUP,SkillCommon_EXP2237,[13]SKILL_0012,[2237]HunterSkill_227,SkillCommon_2237,HunterSkill_227,515


### Specific Skill Detail Relations
#### Source
natives/STM/GameDesign/Common/Equip/SkillData.user.3.json
```
"app.user_data.SkillData": {
    "_Values": [
        {
            "app.user_data.SkillData.cData": {
                "_Index": 3,
                "_dataId": 4,
                "_skillId": "[1]HunterSkill_000",
                "_SkillLv": 3,
                "_skillName": "2a36d1c4-d599-4b2d-8385-47ebef1df235",
                "_skillExplain": "9d878426-10dd-4506-9259-6548d4c59735",
                "_openSkill": [
                    "[1]HunterSkill_000",
                    "[0]NONE",
                    "[0]NONE",
                    "[0]NONE",
                    "[0]NONE"
                ],
                "_value": [
                    100,
                    7,
                    0,
                    0
                ]
            }
        },
```
#### df_spec_skills
|Index|spec_id|common_name|SkillLv|spec_name|description|value|os1|os2|os3|os4|os5|
|-|-|-|-|-|-|-|-|-|-|-|-|
|0|Skill_1|SkillCommon_0|0|Skill_1|Skill_EXP1|[0, 0, 0, 0]|SkillCommon_0|SkillCommon_0|SkillCommon_0|SkillCommon_0|SkillCommon_0|
|1|Skill_2|Attack Boost|1|Attack Boost 1|Attack +3|[100, 3, 0, 0]|Attack Boost|SkillCommon_0|SkillCommon_0|SkillCommon_0|SkillCommon_0|
|2|Skill_3|Attack Boost|2|Attack Boost 2|Attack +5|[100, 5, 0, 0]|Attack Boost|SkillCommon_0|SkillCommon_0|SkillCommon_0|SkillCommon_0|
|3|Skill_4|Attack Boost|3|Attack Boost 3|Attack +7|[100, 7, 0, 0]|Attack Boost|SkillCommon_0|SkillCommon_0|SkillCommon_0|SkillCommon_0|
|4|Skill_5|Attack Boost|4|Attack Boost 4|Attack +2% Bonus: +8|[102, 8, 0, 0]|Attack Boost|SkillCommon_0|SkillCommon_0|SkillCommon_0|SkillCommon_0|


#### Dicts

- dict_backfill = dict(zip(df_1['os1'],df_1['spec_name']))

In [171]:
url = 'https://raw.githubusercontent.com/dtlnor/MHWs-in-json/refs/heads/main/natives/STM/GameDesign/Common/Equip/SkillData.user.3.json'

with urllib.request.urlopen(url) as page:
    raw = json.load(page)
df = pd.json_normalize(raw, record_path=['app.user_data.SkillData', '_Values'])
df.columns = df.columns.str.replace('app.user_data.SkillData.cData._','')

#split openSkill
df_os = df['openSkill'].apply(pd.Series)
df_os.columns = ['os1','os2','os3','os4','os5']

#replace os data
df_os = df_os.replace(dict_common_hunter_name)

#recombine
df = pd.concat([df.drop('openSkill', axis=1), df_os],axis=1)

#data replacement
df['skillName'] = df['skillName'].replace(dict_spec_guid_names)
df['skillExplain'] = df['skillExplain'].replace(dict_spec_guid_desc)
df['skillId'] = df['skillId'].replace(dict_common_hunter_name)

# mask = df['os1'].str.contains('_')
# df_1 = df.loc[mask]
# dict_backfill = dict(zip(df_1['os1'],df_1['spec_name']))

# specific skill name fill blanks with generics
mask = df['skillName'].str.contains('Skill_') & ~df['skillId'].str.contains('SkillCommon_')
df.loc[mask, 'skillName'] = df.loc[mask, 'skillId'] + ' ' + df.loc[mask, 'SkillLv'].astype(str)

#adjust specific skill id
df['dataId'] = 'Skill_' + df['dataId'].astype(str)

df.rename(columns={'dataId':'spec_id',
                   'skillId':'common_name',
                   'skillExplain':'description',
                   'skillName':'spec_name'}, inplace=True)
df_spec_skills = df.copy()
df_spec_skills.reset_index(drop=True, inplace=True)

df_spec_skills.head()
# df_spec_skills.to_excel('specific skills.xlsx')

,Index,spec_id,common_name,SkillLv,spec_name,description,value,os1,os2,os3,os4,os5
0,0,Skill_1,SkillCommon_0,0,Skill_1,Skill_EXP1,"[0, 0, 0, 0]",SkillCommon_0,SkillCommon_0,SkillCommon_0,SkillCommon_0,SkillCommon_0
1,1,Skill_2,Attack Boost,1,Attack Boost 1,Attack +3,"[100, 3, 0, 0]",Attack Boost,SkillCommon_0,SkillCommon_0,SkillCommon_0,SkillCommon_0
2,2,Skill_3,Attack Boost,2,Attack Boost 2,Attack +5,"[100, 5, 0, 0]",Attack Boost,SkillCommon_0,SkillCommon_0,SkillCommon_0,SkillCommon_0
3,3,Skill_4,Attack Boost,3,Attack Boost 3,Attack +7,"[100, 7, 0, 0]",Attack Boost,SkillCommon_0,SkillCommon_0,SkillCommon_0,SkillCommon_0
4,4,Skill_5,Attack Boost,4,Attack Boost 4,Attack +2% Bonus: +8,"[102, 8, 0, 0]",Attack Boost,SkillCommon_0,SkillCommon_0,SkillCommon_0,SkillCommon_0


### Consolidation
#### Backfill common name data

In [172]:
# common names
mask = df_spec_skills['os1'].str.contains('SkillCommon_')
df = df_spec_skills.loc[mask]
dict_ = dict(zip(df['os1'],df['spec_name']))

df_skill_common_ids['name'] = df_skill_common_ids['name'].replace(dict_)
df_skill_common_ids['name'] = df_skill_common_ids['name'].str.replace(' II','')

# forwardfill common skills
dict_ = dict(zip(df_skill_common_ids['common_id'],df_skill_common_ids['name']))
df_common_skills['name'] = df_common_skills['name'].replace(dict_)

# HunterSkill recover
mask = df_spec_skills['os1'].str.contains('HunterSkill_')
df = df_spec_skills.loc[mask]
df = df.iloc[1:]
df = df[['spec_name','os1']]
df.columns = ['name','common_hunter_id']
df['hunter_id'] = df['common_hunter_id'].str.extract(r'^\[[^]]+\](.*)$')
df['common_id'] = 'SkillCommon_' + df['common_hunter_id'].str.extract(r'\[(.*?)\]')
df['common_id'] = df['common_id'].str.replace('-','m')
df['description'] = 'SkillCommon_EXP' + df['common_hunter_id'].str.extract(r'\[(.*?)\]')
df['description'] = df['description'].str.replace('-','m')
df_common_skills = pd.concat([df_common_skills,df], ignore_index=True)
df_common_skills['name'] = df_common_skills['name'].str.replace(' II','')


# df_skill_common_ids
df_common_skills.head()
# df_common_skills.to_excel('common skill.xlsx')

,name,category,description,SkillIconType,common_hunter_id,common_id,hunter_id,SortId
0,Skill_1,[0]EQUIP,SkillCommon_EXP0,[1]SKILL_0000,[0]NONE,SkillCommon_0,NONE,0.0
1,Attack Boost,[3]WEAPON,Increases attack power.,[1]SKILL_0000,[1]HunterSkill_000,SkillCommon_1,HunterSkill_000,1.0
2,Resentment,[0]EQUIP,Increases attack when you have recoverable dam...,[1]SKILL_0000,[1359821952]HunterSkill_060,SkillCommon_1359821952,HunterSkill_060,2.0
3,Adrenaline Rush,[0]EQUIP,Temporarily increases attack power after execu...,[1]SKILL_0000,[1174975744]HunterSkill_115,SkillCommon_1174975744,HunterSkill_115,3.0
4,Offensive Guard,[3]WEAPON,Temporarily increases attack power after execu...,[1]SKILL_0000,[-181127504]HunterSkill_028,SkillCommon_m181127504,HunterSkill_028,4.0


#### Backfill specific name data

In [173]:
dict_ = dict(zip(df_spec_skills['spec_id'],df_spec_skills['spec_name']))
df_skill_spec_ids['name'] = df_skill_spec_ids['name'].replace(dict_)

df_skill_spec_ids.head()

,spec_id,guid_n,name,guid_d,description
0,Skill_1,f7457609-c698-44dc-a87c-3013a81fe34f,Skill_1,3b08702a-e4a1-4de4-b7db-f5897fef1c4b,Skill_EXP1
1,Skill_2,824fa7e2-6344-4f5d-a140-0d411ccc674d,Attack Boost 1,96e65c81-c4a1-4fb0-aea4-c256200cda88,Attack +3
2,Skill_3,c3858b4a-0f53-4739-ade5-ddf1c08497ad,Attack Boost 2,b97ead47-2c69-436f-ab9d-4f9059ec151f,Attack +5
3,Skill_4,2a36d1c4-d599-4b2d-8385-47ebef1df235,Attack Boost 3,9d878426-10dd-4506-9259-6548d4c59735,Attack +7
4,Skill_5,85873022-2c20-4f4a-9e13-6c2e1275918a,Attack Boost 4,86a03b9f-c136-414c-8fe5-501b6c4b6b73,Attack +2% Bonus: +8


#### milestone output

In [ ]:
df_common_skills.to_excel('common skill.xlsx')
df_spec_skills[['spec_id','common_name','SkillLv','spec_name','description']].to_excel('specific skills.xlsx')